This script transforms dataset into the training schema

In [1]:
import pandas as pd
import json
import os
from dataclasses import dataclass
import re

import helper as analytics
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tqdm

In [2]:
def list_checkpoint_folders(directory):
    checkpoint_folders = []
    
    for root, dirs, files in os.walk(directory):
        for folder in dirs:
            if 'checkpoint' in folder:
                checkpoint_folders.append(os.path.join(root, folder))
                
    return checkpoint_folders

In [6]:
CHECKPOINT_FOLDER = "../results/meta-llama/Meta-Llama-3.1-70B-Instruct/small/2024-07-31-09-40-24"
checkpoint_paths = list_checkpoint_folders(CHECKPOINT_FOLDER)

In [7]:
results = []
# get all validation files
for checkpoint_path in checkpoint_paths:
    if not os.path.exists(f"{checkpoint_path}/validation_results.json"):
        print(f"Validation results already exist for {checkpoint_path}")
        continue
    df = pd.read_json(f"{checkpoint_path}/validation_results.json")
    df.loc[df['chatbot_response_clean'] == -1, 'chatbot_response_clean'] = 0
    f1, precision, recall = analytics.calculate_scores(df)
    epoch = analytics.get_epoch_from_checkpoint(checkpoint_paths, int(checkpoint_path.split("/")[-1].replace("checkpoint-", "")))
    result = {
        "checkpoint_path": checkpoint_path,
        "checkpoint_number": checkpoint_path.split("/")[-1].replace("checkpoint-", ""),
        "epoch": epoch,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }
    results.append(result)
df = pd.DataFrame(results)

# sort by highest f1
df_sorted = df.sort_values(by='f1', ascending=False)
df_sorted
df.to_csv(f"{CHECKPOINT_FOLDER}/validation_results.csv")


/ceph/aasteine/fine-tuning-paper/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
def remove_importance_scores(text):
    # Define the regex pattern to match both importance and similarity scores
    pattern = r'\|\|\|importance=[^|]+|\|\|\|similarity=[^|]+'
    
    # Substitute the matches with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

In [8]:
df = pd.read_json('../data/walmart-amazon/walmart-amazon-gs.json.gz', compression='gzip', lines=True)

# rename name_left to title_left and name_right to title_right
df = df.rename(columns={'name_left': 'title_left', 'name_right': 'title_right'})

df.to_pickle('../data/walmart-amazon/walmart-amazon-gs.pkl')

## Explanation shape

In [8]:
df_with_explanation = pd.read_pickle('../data/wdc/preprocessed_wdcproducts80cc20rnd000un_train_small_explanations_40_mini.pkl.gz', compression='gzip')
df_with_explanation["explanation"] = df_with_explanation["explanation"].apply(remove_importance_scores)
df_with_explanation.iloc[0]["explanation"]

'Yes.  \nattribute=brand|||values=Seagate###Seagate|||values=IronWolf Pro|||IronWolf Pro|||values=4TB###4TB|||values=3.5|||3.5|||values=missing###7200RPM|||values=missing###128 MB|||values=SATA3###SATA3'

In [2]:
def create_prompt_with_explanation(product1: str, product2: str, answer: int):
    prompt = f"Do the two product descriptions refer to the same real-world product? Entity 1: '{product1}'. Entity 2: '{product2}'."

    return  {
        "prompt": prompt,
        "completion": answer
    }

In [11]:
df_train = df_with_explanation.apply(lambda row: create_prompt_with_explanation(row["title_left"], row["title_right"], row["explanation"]), axis=1, result_type='expand')

df_train.columns = ['prompt', 'completion']
df_train.to_csv("../data/wdc/explanations/no_importance_similarity/small_simple.csv", index=False)

In [2]:
def create_prompt(product1: str, product2: str, answer: int):
    correct = 'Yes' if answer == 1 else 'No'

    prompt = f"Do the two product descriptions refer to the same real-world product? Entity 1: '{product1}'. Entity 2: '{product2}'."

    return  {
        "prompt": prompt,
        "completion": correct
    }

In [4]:
def create_prompt_row(row):
    answer = row["label"]
    product1 = row["title_left"]
    product2 = row["title_right"]
    correct = 'Yes' if answer == 1 else 'No'

    prompt = f"Do the two entity descriptions refer to the same real-world entity? Entity 1: '{product1}'. Entity 2: '{product2}'."

    return  {
        "prompt": prompt,
        "completion": correct
    }

In [8]:
def convert_row_to_chat_format(row):
    return {
        "messages": [
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['completion']}
        ]
    }


In [13]:
df = pd.read_csv("../data/dblp-scholar/dblp-scholar-valid.csv")

df_train = df.apply(lambda row: create_prompt(f"{row['title_left']}; {row['authors_left']}; {row['venue_left']}; {row['year_left']}", f"{row['title_right']}; {row['authors_right']}; {row['venue_right']}; {row['year_right']}", row["label"]), axis=1, result_type='expand')

df_train.columns = ['prompt', 'completion']
df_train.to_csv("../data/dblp-scholar/dblp-scholar-valid_temp.csv", index=False)

## From batch job


In [5]:
def parse_response(response):
    body = response.get("body", {})
    usage = body.get("usage", {})
    choices = body.get("choices", [{}])
    message = choices[0].get("message", {}) if choices else {}

    return pd.Series({
        "status_code": response.get("status_code"),
        "request_id": response.get("request_id"),
        "completion_id": body.get("id"),
        "created": body.get("created"),
        "model": body.get("model"),
        "content": message.get("content"),
        "prompt_tokens": usage.get("prompt_tokens"),
        "completion_tokens": usage.get("completion_tokens"),
        "total_tokens": usage.get("total_tokens"),
    })

In [11]:
batch_job_df = pd.read_json("../data/wdc/filtered/large/batch_ECDnc1nkc0yQvrwExqFQERpH_output.jsonl", lines=True)
training_df = pd.read_csv("../data/wdc/synthetic/4o/validation/synthetic_examples.csv")

# rename custom_id to pair_id on the training_df
training_df = training_df.rename(columns={'custom_id': 'pair_id'})

# Apply the function to the response column
parsed_df = batch_job_df["response"].apply(parse_response)

# Concatenate the original dataframe with the parsed columns
result_df = pd.concat([batch_job_df, parsed_df], axis=1)

# merge result_df and training_df based on the custom_id from result_df and the index from training_df
result_df = result_df.merge(training_df, left_on="custom_id", right_index=True)

df_train = result_df.apply(
    lambda row: create_prompt_with_explanation(
        row["title_left"],
        row["title_right"],
        f"{'Yes' if row['label'] == 1 else 'No'}. {row['content']}"
    ),
    axis=1,
    result_type='expand'
)

df_train.columns = ['prompt', 'completion']
# Include pair_id in df_train
df_train['pair_id'] = result_df['pair_id']

df_train.to_csv("../data/wdc/synthetic/4o/validation/synthetic_with_explanation.csv", index=False)

ValueError: You are trying to merge on object and int64 columns for key 'custom_id'. If you wish to proceed you should use pd.concat

In [12]:
# split custom_id into prompt, pair_id, label, index
result_df[['task', 'pair_id', 'label', 'index']] = result_df.custom_id.str.split(";", expand=True)

In [14]:
# tanslate content form Yes/No to 1/0
result_df['content'] = result_df['content'].apply(lambda x: 1 if 'Yes' in x else 0)
print(len(result_df))

# convert label to int
result_df['label'] = result_df['label'].astype(int)

# filter out all rows where the content is not equal to the label
result_df = result_df[result_df['content'] == result_df['label']]
print(len(result_df))


19835
19213


In [21]:
train_df = pd.read_pickle("../data/wdc/train_large/preprocessed_wdcproducts80cc20rnd000un_train_large_embeddings.pkl")

# filter out all pair ids that are not in the result_df
train_df = train_df[train_df['pair_id'].isin(result_df['pair_id'])]

#df_expanded = training_df.apply(lambda row: create_prompt(row["title_left"], row["title_right"], row["label"]), axis=1, result_type='expand')
# Rename the columns to match the desired output
#df_expanded.columns = ["prompt", "completion"]

train_df.to_pickle("../data/wdc/filtered/large/filtered_large_embeddings.pkl.gz",  compression='gzip')

In [8]:
df = pd.read_csv("../data/wdc/preprocessed_wdcproducts80cc20rnd000un_train_large_40_mini_explanations.csv")
df

,prompt,completion
0,Do the two entity descriptions refer to the sa...,No. \nattribute=brand|||importance=0.05|||val...
1,Do the two entity descriptions refer to the sa...,Yes. \nattribute=brand|||importance=0.05|||va...
2,Do the two entity descriptions refer to the sa...,Yes. \nattribute=brand|||importance=0.05|||va...
3,Do the two entity descriptions refer to the sa...,Yes. \nattribute=brand|||importance=0.05|||va...
4,Do the two entity descriptions refer to the sa...,No. \nattribute=brand|||importance=0.05|||val...
...,...,...
19830,Do the two entity descriptions refer to the sa...,No. \nattribute=brand|||importance=0.05|||val...
19831,Do the two entity descriptions refer to the sa...,No. \nattribute=brand|||importance=0.05|||val...
19832,Do the two entity descriptions refer to the sa...,No. \nattribute=brand|||importance=0.05|||val...
19833,Do the two entity descriptions refer to the sa...,No. \nattribute=brand|||importance=0.05|||val...


## Synthetic data

In [14]:
def extract_json_objects(data):
    # Navigate to the nested JSON array
    choices = data.get("body", {}).get("choices", [])
    
    if not choices:
        return []

    # Extract the JSON objects as a list
    message_content = choices[0].get("message", {}).get("content", "")
    
    # Find the JSON array in the message content
    start = message_content.find('[')
    end = message_content.rfind(']') + 1
    
    # Extract the JSON array from the string
    json_array_str = message_content[start:end]
    
    # Parse the JSON array
    try:
        json_array = json.loads(json_array_str)
    except json.JSONDecodeError:
        json_array = None
        
    return json_array

In [29]:
# only keep custom_id and entity_one, entity_two, and match columns
synthetic_df = synthetic_df[["custom_id", "entity_one", "entity_two", "match"]]

# convert the match column to a 0 or 1
synthetic_df["match"] = synthetic_df["match"].apply(lambda x: 1 if x == True else 0)

# drop the last two characters from the custom id
synthetic_df["custom_id"] = synthetic_df["custom_id"].apply(lambda x: x[:-2])

# rename entity_one to title_left and entity_two to title_right
synthetic_df = synthetic_df.rename(columns={"entity_one": "title_left", "entity_two": "title_right"})

# rename custom_id to pair_id
synthetic_df = synthetic_df.rename(columns={"custom_id": "pair_id"})

# rename match to label
synthetic_df = synthetic_df.rename(columns={"match": "label"})

synthetic_df.to_csv("../data/wdc/synthetic/4o/textual_explanation/examples.csv", index=False)

In [28]:
synthetic_df = pd.read_json("../data/wdc/synthetic/4o/textual_explanation/batch_M3H2xaBuLy2XyWAIhZSDFMfc_output.jsonl", lines=True)

synthetic_df["response"] = synthetic_df["response"].apply(extract_json_objects)
# drop rows where response is None
synthetic_df = synthetic_df[synthetic_df["response"].notnull()]

# Expand the JSON array into separate rows
synthetic_df = synthetic_df.explode("response")

# split the response into three columns title_left, title_right, label
synthetic_df = pd.concat([synthetic_df.drop(columns=["response"], axis=1), synthetic_df["response"].apply(pd.Series)], axis=1)

# reset the index
synthetic_df.reset_index(drop=True, inplace=True)

#df_train = synthetic_df.apply(lambda row: create_prompt(row["entity_one"], row["entity_two"], row["match"]), axis=1, result_type='expand')

#df_train.columns = ['prompt', 'completion']
df_train.to_csv(f"../data/wdc/synthetic/4o/textual_explanation/synthetic_no_explanations_small_{len(df_train)}.csv", index=False)


In [17]:
# Set the path to your folder containing the JSON files
folder_path = '../data/wdc/explanations'

# List all JSON files in the directory
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# Read each JSON file and store in a list of DataFrames
dataframes = [pd.read_json(os.path.join(folder_path, file)) for file in json_files]

# Concatenate all the DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Now you can use combined_df as your single DataFrame containing all the data
print(combined_df.head())  # Display the first few rows to check


          task                                   chatbot_question  \
0  explanation  Do the two entity descriptions refer to the sa...   
1  explanation  Do the two entity descriptions refer to the sa...   
2  explanation  Do the two entity descriptions refer to the sa...   
3  explanation  Do the two entity descriptions refer to the sa...   
4  explanation  Do the two entity descriptions refer to the sa...   

                                chatbot_response_raw  chatbot_response_clean  \
0  Yes, the two entity descriptions refer to the ...                       1   
1  Yes, the two entity descriptions refer to the ...                       1   
2  Yes, the two entity descriptions refer to the ...                       1   
3  Yes, the two entity descriptions refer to the ...                       1   
4  No, the two entity descriptions do not refer t...                       0   

    id_left brand_left                                         title_left  \
0  87617353        NaN     

In [19]:
unique_df = combined_df.drop_duplicates(subset='pair_id', keep='first')

In [24]:
# Apply the function and expand the dictionary into separate columns
df_expanded = unique_df.apply(lambda row: create_prompt(row["title_left"], row["title_right"], row["label"]), axis=1, result_type='expand')

# Rename the columns to match the desired output
df_expanded.columns = ["prompt", "completion"]

#df_expanded.to_csv("../data/wdc/explanations/domain_complex_free_with_explanation.csv", index=False)

In [46]:
# drop task, chatbot_response_raw, and chatbot_response_clean
df_explanations = df_explanations.drop(['task', 'chatbot_response_raw', 'chatbot_response_clean', 'chatbot_question'], axis=1)

In [52]:
df_explanations["chatbot_response_raw"].iloc[0]

'Yes, the two entity descriptions refer to the same real-world entity, which is the Sony FE 24-240mm f3.5-6.3 OSS lens.\n\nThe first entity description uses the format "Brand Name Model Name" and includes the manufacturer\'s name, Sony, followed by the model name, FE 24-240mm F/3.5-6.3 OSS.\n\nThe second entity description uses a more simplified format and omits the brand name, instead focusing on the model name, FE 24-240mm f3.5-6.3 OSS.\n\nBoth entity descriptions refer to the same lens, which is a telephoto zoom lens produced by Sony for their full-frame mirrorless cameras. The lens has a focal length range of 24-240mm and a maximum aperture of f3.5-6.3, with optical image stabilization (OSS) technology.\n\nTherefore, the two entity descriptions are equivalent and refer to the same real-'

In [48]:
df_explanations.to_csv("../data/wdc/explanations/4k_ralph.csv", index=False)

## Chat format

### Testset

In [15]:
file_path = "../data/dblp-scholar/dblp-scholar-train.csv"
df_to_convert = pd.read_csv(file_path)

df_to_convert = df_to_convert.apply(convert_row_to_chat_format, axis=1)
# Convert the DataFrame to a list of dictionaries
records = df_to_convert.to_list()

# Save each record as a separate JSON object in a .jsonl file
with open(f"{file_path.split('.csv')[0]}_gpt.jsonl", 'w') as f:
    for record in records:
        f.write(json.dumps(record) + '\n')

### Validation set

In [39]:
validation_file_path = "../data/wdc/preprocessed_wdcproducts80cc20rnd000un_valid_small.csv"    
valdiation_df = pd.read_csv(validation_file_path)

valdiation_df = valdiation_df.apply(create_prompt_row, axis=1, result_type='expand')

df_to_convert = valdiation_df.apply(convert_row_to_chat_format, axis=1)
# Convert the DataFrame to a list of dictionaries
records = df_to_convert.to_list()

# Save each record as a separate JSON object in a .jsonl file
with open(f"{validation_file_path.split('.csv')[0]}_gpt.jsonl", 'w') as f:
    for record in records:
        f.write(json.dumps(record) + '\n')

KeyError: 'label'

## Simple Fine-tuning

In [4]:
df = pd.read_csv("../data/wdc/filtered/wdc_small_filtered.csv")
df = df.apply(create_prompt_row, axis=1, result_type='expand')
df.to_csv("../data/wdc/filtered/wdc_train_small_filtered.csv", index=False)

## Create validation

In [3]:
validation_df = pd.read_csv("../data/walmart-amazon/walmart-amazon-valid.csv")

In [4]:
train_df = pd.read_json("../data/walmart-amazon/walmart-amazon-train.json.gz", compression='gzip', lines=True)
# Filter df1 where 'pair_id' is in df2['pair_id']
filtered_df = train_df[train_df['pair_id'].isin(validation_df['pair_id'])]
filtered_df.to_csv("../data/walmart-amazon/walmart-amazon-valid.csv", index=False)

In [8]:
# Apply the function and expand the dictionary into separate columns
df_expanded = df.apply(lambda row: create_prompt(row["title_left"], row["title_right"], row["label"]), axis=1, result_type='expand')

# Rename the columns to match the desired output
df_expanded.columns = ["prompt", "completion"]

df_expanded.to_csv("../data/wdc/preprocessed_wdcproducts80cc20rnd000un_train_medium_domain_complex_force.csv", index=False)

In [27]:
# Applying the conversion to each row and saving the results
with open('../data/wdc/preprocessed_wdcproducts80cc20rnd000un_valid_large_domain_complex_free.jsonl', 'w') as f:
    for index, row in df_expanded.iterrows():
        chat_format = convert_row_to_chat_format(row)
        f.write(json.dumps(chat_format) + '\n')

In [15]:
df_expanded.to_json("../data/wdc/preprocessed_wdcproducts80cc20rnd000un_valid_large_domain_complex_free.json", orient='records', lines=True)

In [6]:
df["prompt"] = df.apply(lambda row: create_prompt(row["title_left"], row["title_right"], row["label"]), axis=1)